## Transactions
#### 03.4 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2021-12-19

See also: https://developer.algorand.org/tutorials/creating-python-transaction-purestake-api/

- Load credentials
- Create our own QR code for payments
- Interact with the blockchain and execute a payment from Python

## Setup
Starting with this chapter 3.4, the lines below will always automatically load ...
* The accounts MyAlgo, Alice, Bob, Charlie, Dina
* The Purestake credentials
* The functions in `algo_util.py`

In [ ]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future.transaction import PaymentTxn, MultisigTransaction
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json

In [ ]:
print(MyAlgo['public'])
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])
print(Dina['public'])

### Check the accounts on the blockchain
- Go to https://algoexplorer.io and insert address
- Go to https://testnet.algoexplorer.io for the testnet

In [ ]:
# Create a link to directly access your MyAlgo account
print('https://algoexplorer.io/address/'+MyAlgo['public'])
print('https://testnet.algoexplorer.io/address/'+MyAlgo['public'])

### Fund with testnet Algos
- https://bank.testnet.algorand.network/
- https://testnet.algoexplorer.io/dispenser
- Fund all three accounts. How many test ALGOs did you get?

## Connecting Python to the Algorand Blockchain
Options:
- Set up your own indexer
- Set up your own virtual indexer using Docker
- Use a third party API ... we use Purestake

### Purestake token for authenticate
- See 03.3_WSC_Credentials
- API cendentials stored in `cred['purestake_token']`
  - Note: this is already the pair `{'X-Api-key': '(your token'}`
  - To obtain token alone `cred['purestake_token']['X-Api-key']`

In [ ]:
algod_token   = ''                        # Only needed if we have our own server, 
algod_address = cred['algod_test']        # Or cred['algod_main'] 
purestake_token = cred['purestake_token'] # Authentication token pair {'X-Api-key': '(your token'}

# Initialize the algod client
algod_client = algod.AlgodClient(algod_token=algod_token, algod_address=algod_address, headers=purestake_token)

#### Test the connection
- Our first Python access of the blockchain
- What's the last block?
- Check on https://testnet.algoexplorer.io
- Note that block count on testnet is larger (Why?)

In [ ]:
algod_client.status()["last-round"]

### Obtain holdings

In [ ]:
# Get holdings of testnet Algos
address = Alice["public"]
algod_client.account_info(address)["amount"]

In [ ]:
# Holdings are in micro Algo ... convert
algo_precision = 1e6
algo_amount    = algod_client.account_info(address)["amount"]/algo_precision
print(f"Address {address} has {algo_amount} test algos")

#### Suggested parameters for a transaction (on the test network)

In [ ]:
sp = algod_client.suggested_params()
print(json.dumps(vars(sp), indent=4))

## A first payment transaction

#### Step 1: prepare and create unsigned transaction

In [ ]:
# Parameters
sp       = algod_client.suggested_params()       # suggested params
amount   = 0.1
algo_precision = 1e6
amt_microalgo = int(amount * algo_precision)

# Create (unsigned) TX
txn = PaymentTxn(sender = Alice['public'],     # <--- From
                 sp = sp, 
                 receiver = Bob['public'],     # <---- To
                 amt = amt_microalgo           # <---- Amount in Micro-ALGOs
                )
print(txn)
print(txn.get_txid())

In [ ]:
# Is it already on the blockchain? No ... not yet sent
print('https://testnet.algoexplorer.io/tx/'+txn.get_txid())

#### Step 2: sign

In [ ]:
stxn = txn.sign(Alice['private'])                 # <---- Alice signs with private key

# Transaction ID is the same, but still nothing on the blockchain
print('https://testnet.algoexplorer.io/tx/'+stxn.get_txid())

#### Step 3: send

In [ ]:
txid = algod_client.send_transaction(stxn)
print("Send transaction with txID: "+txid)

# The freshly submitted transaction on the blockchain
txinfo = algod_client.pending_transaction_info(txid)
print(txinfo)

#### Step 4: Wait for confirmation

In [ ]:
txinfo = wait_for_confirmation(algod_client, txid)

In [ ]:
# Note that txinfo has now a 'confirmed-round'
print(txinfo)
print('https://testnet.algoexplorer.io/tx/'+txid)

### Add a note to a transaction

In [ ]:
# Step 1a: Prepare
sp       = algod_client.suggested_params()       # suggested params

amount    = 0.1
algo_precision = 1e6
amt_microalgo = int(amount * algo_precision)

# Step 1b: The note
# Start with a Python dict, create JSON, byte-encode
note_dict = {"Message":"Paying back for last dinner", "From":"Alice", "To":"Bob"}
note_json = json.dumps(note_dict)
note_byte = note_json.encode() 

In [ ]:
# Step 1c: create (unsigned) TX
txn = PaymentTxn(sender=Alice['public'],
                 sp=sp, 
                 receiver = Bob['public'],
                 amt=amt_microalgo, 
                 note=note_byte
                 )
print(txn)

In [ ]:
# Step 2+3: sign and send TX
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)
print("Send transaction with txID: {}".format(txid))

In [ ]:
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

In [ ]:
print("https://testnet.algoexplorer.io/tx/"+txid)

In [ ]:
print(txinfo)

In [ ]:
# Convert message in txinfo to Python dict
import base64
message_base64 = txinfo['txn']['txn']['note']
print(message_base64)
message_byte   = base64.b64decode(message_base64)
print(message_byte)
message_json   = message_byte.decode()
print(message_json)
message        = json.loads( message_json )
print( message['From'] )

## Useful functions
These function `wait_for_confirmation` is actually not an ufficial Algorand function.<br>
Below is the source code.

In [ ]:
def wait_for_confirmation(client, txid):
    # client = algosdk client
    # txid = transaction ID, for example from send_payment()

    txinfo = client.pending_transaction_info(txid)       # obtain transaction information
    current_round = algod_client.status()["last-round"]        # obtain last round number
    print("Current round is  {}.".format(current_round))
    
    # Wait for confirmation
    while ( txinfo.get('confirmed-round') is None ):            # condition for waiting = 'confirmed-round' is empty
        print("Waiting for round {} to finish.".format(current_round))
        algod_client.status_after_block(current_round)             # this wait for the round to finish
        txinfo = algod_client.pending_transaction_info(txid)    # update transaction information
        current_round += 1

    print("Transaction {} confirmed in round {}.".format(txid, txinfo.get('confirmed-round')))
    return txinfo

## Also useful functions
These functions are much more convenient:
- `note_encode` encodes a note from a Pyhon dict
- `note_decode` decodes a note into a Pyhon dict

In [ ]:
def note_encode(note_dict):
    # note dict ... a Python dictionary
    note_json = json.dumps(note_dict)
    note_byte = note_json.encode()     
    return(note_byte)

def note_decode(note_64):
    # note64 =  note in base64 endocing
    # returns a Python dict
    import base64
    message_base64 = txinfo['txn']['txn']['note']
    message_byte   = base64.b64decode(message_base64)
    message_json   = message_byte.decode()
    message_dict   = json.loads( message_json )
    return(message_dict)

## Exercise
* Send 0.8 ALGO from Dina to Charlie with a thank you note

In [ ]:
# Your Python code goes here

## Things that do not and will not work
Let's produce some error messages. Following are a few things that don't work

In [ ]:
# Need to import this to be able to read error messages
import sys, algosdk.error

### Overspending
Alice sends more than she owns

In [ ]:
# Step 1: prepare
sp       = algod_client.suggested_params()
algo_precision = 1e6
sender   = Alice['public']
receiver = Bob['public']
amount   = 100                       # <----------------- way too much!
amount_microalgo = int(amount * algo_precision)

# Step 2: create unsigned TX
unsigned_txn = PaymentTxn(sender, sp, receiver, amount_microalgo)

# Step 3a: Sign
signed_txn = unsigned_txn.sign(Alice['private'])

In [ ]:
# Step 3b: Send
txid = algod_client.send_transaction(signed_txn)

#### Can we *catch the error* and get a better structured error message?

In [ ]:
# Step 1: prepare
sp       = algod_client.suggested_params()
algo_precision = 1e6
sender   = Alice['public']
receiver = Bob['public']
amount   = 100                       # <----------------- way too much!
amount_microalgo = int(amount * algo_precision)

# Step 2: create unsigned TX
unsigned_txn = PaymentTxn(sender, sp, receiver, amount_microalgo)

# Step 3a: Sign
signed_txn = unsigned_txn.sign(Alice['private'])

In [ ]:
# Step 3b: Send
try:
    txid = algod_client.send_transaction(signed_txn)
except algosdk.error.AlgodHTTPError as err:
    print(err)                                   # print entire error message
    if ("overspend" in str(err)):                # check for specific type of error
        print("Overspend error")         
    txid = None

#### What happens if we wait for the failed transaction to complete?

In [ ]:
# We fail at the first command
try:
    txinfo = algod_client.pending_transaction_info(txid)       # obtain transaction information
    print(txinfo)
except TypeError as err:                    # obtain error message
    # print entire error message
    print(err)
    # check for specific type of error
    print("txid is empty")

### Wrong signature
Bob tries to sign a transaction from Alice to Bob

In [ ]:
# Step 1: prepare
sp       = algod_client.suggested_params()
algo_precision = 1e6
sender   = Alice['public']
receiver = Bob['public']
amount   = 0.1
amount_microalgo = int(amount * algo_precision)

# Step 2: create unsigned TX
unsigned_txn = PaymentTxn(sender, sp, receiver, amount_microalgo)

# Step 3a: Sign
signed_txn = unsigned_txn.sign(Bob['private'])                        # <----------------- wrong person signs!

try:
    txid = algod_client.send_transaction(signed_txn)
except algosdk.error.AlgodHTTPError as err:
    # print entire error message
    print(err)
    if ("should have been authorized" in str(err)):                # check for specific type of error
        print("Wrong signature error")         
    txid = None

### Sending the *indentical* transaction twice
* "Identical" means same ...
    * Sender
    * Recipien
    * Ammount
    * Parameters

In [ ]:
# Step 1: prepare
sp       = algod_client.suggested_params()
algo_precision = 1e6
sender   = Alice['public']
receiver = Bob['public']
amount   = 0.1
amount_microalgo = int(amount * algo_precision)

In [ ]:
# Step 2: create unsigned TX
unsigned_txn = PaymentTxn(sender, sp, receiver, amount_microalgo)

# Step 3: Sign and send
signed_txn = unsigned_txn.sign(Alice['private'])
try:
    txid = algod_client.send_transaction(signed_txn)
    print("Submitted with txID: {}".format(txid))
except algosdk.error.AlgodHTTPError as err:
    # print entire error message
    print(err)
    if ("transaction already in ledger" in str(err)):                # check for specific type of error
        print("Identical transaction {} has been submitted twice.".format(signed_txn.get_txid()))         
    txid = None    # check for specific type of error

**REPEAT** only step 2-3 $\rightarrow$ error message<br>
**REPEAT** only step 1-3 $\rightarrow$ no error <br>

#### See how the sp change
* Re-run this after 2-3 seconds

In [ ]:
sp = algod_client.suggested_params()
print(json.dumps(vars(sp), indent=4))